<a href="https://colab.research.google.com/github/Lambda-School-Labs/Labs26-Citrics-DS-TeamC/blob/mtoce/notebooks/Covid_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COVID-19 Notebook for Citrics Team C (Driftly)

# Install Packages

In [1]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import time
import datetime
import glob     # loading in .csvs
# import plotly.express as px
# from sodapy import Socrata    # for querying the API

In [2]:
# MOUNT google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Query the Data

- Downloaded all 50 states of data from [Coronavirus API](https://coronavirusapi.com/) using the link at the bottom to query for each state based on it's postal code.

- Link to query the data: http://coronavirusapi.com/getTimeSeries/(2 letter state abbreviation)

In [3]:
# LOAD in postal's dict
postals_dict = {'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR','California': 'CA','Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE','District of Columbia': 'DC','Florida': 'FL','Georgia': 'GA','Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Iowa': 'IA','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS','Missouri': 'MO','Montana': 'MT','Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM','New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH','Oklahoma': 'OK','Oregon': 'OR','Pennsylvania': 'PA','Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Vermont': 'VT','Virginia': 'VA','Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY'}
postals_list = list(postals_dict.values())
print(len(postals_list))

51


In [24]:
def clean(df):
  '''
  Converts seconds since epoch into the actual date and creates year, month, and
  day columns from date.
  '''
  # create date col from seconds since epoch col
  df['date'] = pd.to_datetime(df['seconds_since_Epoch'], unit='s')
  # create year, month, day cols from date col
  df['year'] = pd.DatetimeIndex(df['date']).year
  df['month'] = pd.DatetimeIndex(df['date']).month
  df['day'] = pd.DatetimeIndex(df['date']).day
  return df

In [25]:
# FUNCTIONALIZE querying the data so we can grab all 50 states' data at what intervals we want
def query_and_store_covid_data():
  '''
  Loops through all state postal codes, querying the coronavirus API or each one.
  Converts each to a dataframe, cleans it, and exports to a new csv.
  '''
  # LOOP over all state-postals
  for idx in range(len(postals_list)):
    base_url = 'http://coronavirusapi.com/getTimeSeries/'
    r = requests.get(base_url + postals_list[idx])    # base url + postal code
    bytes_data = r.content                # stores bytes data to variable
    s = str(bytes_data, 'utf-8')          # convert bytes data to a usable string
    data = StringIO(s)
    df = pd.read_csv(data)                # read in object as a dataframe
    df['postal'] = postals_list[idx]      # create 'postal' column
    df = clean(df)
    # SAVE to .csv inside google drive folder
    df.to_csv('/content/drive/My Drive/lambda_school/driftly_data/covid_data/covid_data_updated/' + postals_list[idx] + '.csv')
query_and_store_covid_data()

In [33]:
# LOAD in .CSVs from google collab folder
load_path = r'/content/drive/My Drive/lambda_school/driftly_data/covid_data/covid_data_updated'
export_path = r'/content/drive/My Drive/lambda_school/driftly_data/covid_data/covid_data.csv'
def load_and_concat(load_path, export_path):
  all_files = glob.glob(path + "/*.csv")
  list_to_concat = []
  # Loop over all files in folder
  for filename in all_files:
      df = pd.read_csv(filename, index_col=None, header=0)
      list_to_concat.append(df)
  # concatenate .csvs into one DF
  df = pd.concat(list_to_concat, axis=0, ignore_index=True)
  df = df.drop('Unnamed: 0', axis=1)
  df.to_csv(export_path)
  return df
df = load_and_concat(load_path, export_path)
df.head()

,seconds_since_Epoch,tested,positive,deaths,postal,date,year,month,day
0,1588293736,19562,355,9,AK,2020-05-01 00:42:16,2020,5,1
1,1588372862,19562,364,9,AK,2020-05-01 22:41:02,2020,5,1
2,1588390979,20325,364,9,AK,2020-05-02 03:42:59,2020,5,2
3,1588445522,20325,365,9,AK,2020-05-02 18:52:02,2020,5,2
4,1588466696,21399,365,9,AK,2020-05-03 00:44:56,2020,5,3


In [43]:
# CONVERT dataframe to JSON object
def create_covid_json(df):
  grpd_avg = df.groupby('postal').mean()
  grpd_avg = grpd_avg.drop(['seconds_since_Epoch','year', 'month', 'day'], axis=1)
  avg_json = grpd_avg.to_json(orient='index')
  return avg_json
avg_json = create_covid_json(df)
avg_json

'{"AK":{"tested":101194.1470588235,"positive":1030.2941176471,"deaths":14.2941176471},"AL":{"tested":358771.4265402844,"positive":44907.5687203791,"deaths":994.8293838863},"AR":{"tested":229584.2753623188,"positive":16518.652173913,"deaths":235.6666666667},"AZ":{"tested":721558.1752577319,"positive":93395.7628865979,"deaths":2182.3917525773},"CA":{"tested":6022203.9189189188,"positive":360479.2972972973,"deaths":7693.7027027027},"CO":{"tested":335332.9649122807,"positive":34315.9736842105,"deaths":1463.5614035088},"CT":{"tested":394123.7794117647,"positive":42083.2352941177,"deaths":3795.2941176471},"DC":{"tested":135301.3969465649,"positive":10529.4885496183,"deaths":518.0992366412},"DE":{"tested":87644.78125,"positive":10107.75,"deaths":386.40625},"FL":{"tested":2619268.1739130435,"positive":308108.0289855073,"deaths":5898.6956521739},"GA":{"tested":723217.5974842767,"positive":81104.2955974843,"deaths":2327.9245283019},"HI":{"tested":70864.5949367089,"positive":1375.2151898734,"deat